# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

Park Restaurant.

In this project we will try to find an optimal location for a regular park restaurant. Specifically, this report will be targeted on the parks that's with less restaurants arrount then.

Since there are lots of restaurants in Toronto we will try to detect **park locations that are not already crowded with restaurants**.

We will use our data science powers to generate a few most promissing parks  based.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants arrount the parks
* number of parks near the promissor point.

We decided to use regularly spaced grid of locations, centered around city center.

Following data sources will be needed to extract/generate the required information:
* Extract the lists of ceps of Toronto using the webscraping of wikipedia
* Get the longitude and latitude
* Use foursquare to get venues
* Filter parks
* Filter the restaurants
* Generate a map showing a parks and restaurants near then


Analysis of Toronto

Webscrap and create a dataframe

In [138]:
#installs

!pip install geocoder

In [139]:

#imports
import numpy as np # para a manipulação de vetores
import pandas as pd # para a analise de dados
import json # para manipulação de .json

# para transformar um endereço em latitude e longitude
from geopy.geocoders import Nominatim 
# manipulação de requests
import requests 
# para transformar um .json em um dataframe pandas
from pandas.io.json import json_normalize 
# Matplot para criação dos graficos
import matplotlib.cm as cm
import matplotlib.colors as colors
# Sklearning usado para machine learning Kmeans para agrupamento
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#para a geranção de mapas
import folium
from bs4 import BeautifulSoup
import lxml
#imput output
import io
#importar geocoder
import geocoder 



#remover limitador para visualizacao
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('ok')

ok


In [140]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())


In [141]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#________________________________________________________________________________________________
#Delete rows where Borough is 'Not assigned'
df = df[df.Borough!= "Not assigned"]

#Combine all neighbourhoods where postcode and Borough are the same
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
#if Neighbourhood is not assigned, replace by borough name
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood']

# instantiate the dataframe
df.columns = column_names

print(df.to_markdown())

|     | PostalCode   | Borough                | Neighborhood                                                                                                                           |
|----:|:-------------|:-----------------------|:---------------------------------------------------------------------------------------------------------------------------------------|
|   0 | M1B          | Scarborough            | Malvern, Rouge                                                                                                                         |
|   1 | M1C          | Scarborough            | Rouge Hill, Port Union, Highland Creek                                                                                                 |
|   2 | M1E          | Scarborough            | Guildwood, Morningside, West Hill                                                                                                      |
|   3 | M1G          | Scarborough            | Woburn                     

Insert longitude and lattitute in dataframe

In [142]:
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

dfc = df.join(c.set_index('Postal Code'), on='PostalCode')
dfc.head()
print(dfc.to_markdown())

|     | PostalCode   | Borough                | Neighborhood                                                                                                                           |   Latitude |   Longitude |
|----:|:-------------|:-----------------------|:---------------------------------------------------------------------------------------------------------------------------------------|-----------:|------------:|
|   0 | M1B          | Scarborough            | Malvern, Rouge                                                                                                                         |    43.8067 |    -79.1944 |
|   1 | M1C          | Scarborough            | Rouge Hill, Port Union, Highland Creek                                                                                                 |    43.7845 |    -79.1605 |
|   2 | M1E          | Scarborough            | Guildwood, Morningside, West Hill                                                                       

Show neighborhoods in Toronto

In [143]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto coordinates {}, {}.'.format(latitude, longitude))

Toronto coordinates 43.6534817, -79.3839347.


In [144]:
#map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

In [145]:
#insert markers

for lat, lng, borough, neighborhood in zip(dfc['Latitude'], dfc['Longitude'], dfc['Borough'], dfc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [146]:
#Fousquare
CLIENT_ID = 'BNIAVGNS2GGN3WW4RLKOA2FTWFWTLT3WBFMN1ZU2F1U2FBD5' # your Foursquare ID
CLIENT_SECRET = 'SJJFOWFEIGF5KN2IWB14G5ANP4NHB5QOCKZTBOQLNCINVLOL' # your Foursquare Secret
ACCESS_TOKEN = 'DSAIWWVJH3TYFYFXICF3YCAG0KOPW2MQHUE5F0NSNWMXHRGV' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


#Send the GET request and examine the results
results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

Your credentails:
CLIENT_ID: BNIAVGNS2GGN3WW4RLKOA2FTWFWTLT3WBFMN1ZU2F1U2FBD5
CLIENT_SECRET:SJJFOWFEIGF5KN2IWB14G5ANP4NHB5QOCKZTBOQLNCINVLOL


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,5b193c42598e64002ca79b96,City of Toronto Civic Innovation Office,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1621540525,False,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN
1,4ad4c05ef964a5208ff620e3,Toronto City Hall,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1621540525,False,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.
2,4c41d47cd691c9b6fa4c8d0a,Kew Gardens Playground,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",v-1621540525,False,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN
3,4c0121fd9a950f47fa9208c6,City Hall Podium Green Roof,"[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",v-1621540525,False,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN
4,4e4a878c52b11cf651b30044,Mayor Office Toronto,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1621540525,False,100 Queen St. West,43.653367,-79.384021,"[{'label': 'display', 'lat': 43.653367, 'lng':...",14,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. West, Toronto ON M5H 2N2, Canada]",NaN


Make analysis of Venues Data Frame


In [147]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,5b193c42598e64002ca79b96
1,Toronto City Hall,City Hall,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.,4ad4c05ef964a5208ff620e3
2,Kew Gardens Playground,Playground,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN,4c41d47cd691c9b6fa4c8d0a
3,City Hall Podium Green Roof,Garden,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN,4c0121fd9a950f47fa9208c6
4,Mayor Office Toronto,Office,100 Queen St. West,43.653367,-79.384021,"[{'label': 'display', 'lat': 43.653367, 'lng':...",14,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. West, Toronto ON M5H 2N2, Canada]",NaN,4e4a878c52b11cf651b30044


In [148]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,5b193c42598e64002ca79b96
1,Toronto City Hall,City Hall,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.,4ad4c05ef964a5208ff620e3
2,Kew Gardens Playground,Playground,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN,4c41d47cd691c9b6fa4c8d0a
3,City Hall Podium Green Roof,Garden,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN,4c0121fd9a950f47fa9208c6
4,Mayor Office Toronto,Office,100 Queen St. West,43.653367,-79.384021,"[{'label': 'display', 'lat': 43.653367, 'lng':...",14,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. West, Toronto ON M5H 2N2, Canada]",NaN,4e4a878c52b11cf651b30044


In [149]:
#Create function to know how many venues there are in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)




#List of Neighborhood that have venues in Toronto
toronto_venues = getNearbyVenues(names=dfc['Neighborhood'],
                                   latitudes=dfc['Latitude'],
                                   longitudes=dfc['Longitude']
                                  )

toronto_venues.head()

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [150]:

#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,30,30,30,30,30,30
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17


In [151]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [152]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [153]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

----Agincourt----
----Alderwood, Long Branch----
----Bathurst Manor, Wilson Heights, Downsview North----
----Bayview Village----
----Bedford Park, Lawrence Manor East----
----Berczy Park----
----Birch Cliff, Cliffside West----
----Brockton, Parkdale Village, Exhibition Place----
----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
----Caledonia-Fairbanks----
----Cedarbrae----
----Central Bay Street----
----Christie----
----Church and Wellesley----
----Clairville, Humberwood, Woodbine Downs, West Humber, Kipling Heights, Rexdale, Elms, Tandridge, Old Rexdale----
----Clarks Corners, Tam O'Shanter, Sullivan----
----Cliffside, Cliffcrest, Scarborough Village West----
----Commerce Court, Victoria Hotel----
----Davisville----
----Davisville North----
----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
----Don Mills North----
----Don Mills South----
----Dorset Park, Wexford Heights, Scarborough Town Centre----
----Downsview Ce

In [154]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Chinese Restaurant
1,"Alderwood, Long Branch",Pizza Place,Playground,Sandwich Place,Pharmacy
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pizza Place,Ice Cream Shop
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant
4,"Bedford Park, Lawrence Manor East",Pizza Place,Coffee Shop,Sandwich Place,Restaurant


In [155]:

# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = dfc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged.dropna()


toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Yoga Studio,Deli / Bodega
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Bar,Yoga Studio,Falafel Restaurant,Escape Room
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Breakfast Spot,Medical Center,Electronics Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Pharmacy,Indian Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Gas Station,Bank,Bakery


In [156]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [157]:
toronto_merged.loc[toronto_merged['ClusterLabels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
14,Scarborough,1.0,Playground,Intersection,Park,Escape Room
21,North York,1.0,Park,Yoga Studio,Deli / Bodega,Escape Room
23,North York,1.0,Electronics Store,Park,Convenience Store,Yoga Studio
25,North York,1.0,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio
30,North York,1.0,Airport,Park,Yoga Studio,Deli / Bodega
40,East York/East Toronto,1.0,Park,Convenience Store,Yoga Studio,Deli / Bodega
44,Central Toronto,1.0,Park,Swim School,Bus Line,Yoga Studio
48,Central Toronto,1.0,Park,Tennis Court,Yoga Studio,Deli / Bodega
50,Downtown Toronto,1.0,Park,Playground,Trail,Dance Studio
74,York,1.0,Park,Women's Store,Pool,Distribution Center


In [158]:
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Yoga Studio,Deli / Bodega
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Bar,Yoga Studio,Falafel Restaurant,Escape Room
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Intersection,Breakfast Spot,Medical Center,Electronics Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean BBQ Restaurant,Pharmacy,Indian Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Gas Station,Bank,Bakery
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0.0,Playground,Gym / Fitness Center,Escape Room,Electronics Store
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,0.0,Coffee Shop,Hobby Shop,Discount Store,Department Store
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,0.0,Bakery,Intersection,Bus Station,Soccer Field
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,0.0,Skating Rink,American Restaurant,Motel,Intersection
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0.0,General Entertainment,Skating Rink,Café,College Stadium


#Methodology

In this project we will direct our efforts on detecting parks of Toronto  that have low restaurant density, particularly those with low number of regular restaurants and fast food restaurants.

First we have colected the location of Parks, then collect the location of regular restaurants and last the fast food restaurants

After this  we locate the most promissing areas with less restaurants.

# Analysis

Analysis of parks

Now we find the best parks to put on restaurante, fiding the park with less restaurants arrount.

First we filter the parks and color them in green

In [159]:
parks = toronto_venues.loc[toronto_venues['Venue Category'] =='Park']

parks.head()
parks = parks.reset_index()
print(parks.head())

   index                                       Neighborhood  \
0     37                    Golden Mile, Clairlea, Oakridge   
1     79  Milliken, Agincourt North, Steeles East, L'Amo...   
2    132                            Willowdale, Newtonbrook   
3    164                                    York Mills West   
4    171                                          Parkwoods   

   Neighborhood Latitude  Neighborhood Longitude              Venue  \
0              43.711112              -79.284577  Warden Woods Park   
1              43.815252              -79.284577    Port Royal Park   
2              43.789053              -79.408493   Newtonbrook Park   
3              43.752758              -79.400049    Tournament Park   
4              43.753259              -79.329656    Brookbanks Park   

   Venue Latitude  Venue Longitude Venue Category  
0       43.710527       -79.278966           Park  
1       43.815477       -79.289773           Park  
2       43.786942       -79.410022    

In [160]:


map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

mapanalysis01 = folium.Map(
    location=[43.646435, -79.374846],    # Coordenadas retiradas do Google Maps
    zoom_start=12
)


for x in range(len(parks['Venue'])):
  lat = parks['Venue Latitude'][x]
  lon = parks['Venue Longitude'][x]
  folium.Marker(location=[lat, lon],icon=folium.Icon(color='green')).add_to(mapanalysis01)



mapanalysis01


Analysis of restaurants

Then we filter the regular restaurants and color them in red

In [161]:
restaurants = toronto_venues.loc[toronto_venues['Venue Category'] =='Restaurant']

restaurants.head()
restaurants = restaurants.reset_index()
print(restaurants.head())

   index                       Neighborhood  Neighborhood Latitude  \
0      5  Guildwood, Morningside, West Hill              43.763573   
1    100       Fairview, Henry Farm, Oriole              43.778517   
2    119       Fairview, Henry Farm, Oriole              43.778517   
3    153                   Willowdale South              43.770120   
4    183                    Don Mills South              43.725900   

   Neighborhood Longitude                               Venue  Venue Latitude  \
0              -79.188711                          Sail Sushi       43.765951   
1              -79.346556      New York Fries - Fairview Mall       43.778605   
2              -79.346556                        Thai Express       43.777990   
3              -79.408493  Symposium Cafe Restaurant & Lounge       43.771075   
4              -79.340923                        Swiss Chalet       43.726747   

   Venue Longitude Venue Category  
0       -79.191275     Restaurant  
1       -79.343577  

In [162]:
for x in range(len(restaurants['Venue'])):
  lat = restaurants['Venue Latitude'][x]
  lon = restaurants['Venue Longitude'][x]
  folium.Marker(location=[lat, lon],icon=folium.Icon(color='red')).add_to(mapanalysis01)
  


mapanalysis01

Analysis of fast food restaurants

Now we filter the fast food restaurant and color them in orange

In [163]:
fastFoodRestaurants = toronto_venues.loc[toronto_venues['Venue Category'] =='Fast Food Restaurant']

fastFoodRestaurants.head()
fastFoodRestaurants = fastFoodRestaurants.reset_index()
print(fastFoodRestaurants.head())

   index                             Neighborhood  Neighborhood Latitude  \
0      0                           Malvern, Rouge              43.806686   
1     73  Clarks Corners, Tam O'Shanter, Sullivan              43.781638   
2     75  Clarks Corners, Tam O'Shanter, Sullivan              43.781638   
3     84            Steeles West, L'Amoreaux West              43.799525   
4     88            Steeles West, L'Amoreaux West              43.799525   

   Neighborhood Longitude       Venue  Venue Latitude  Venue Longitude  \
0              -79.194353     Wendy’s       43.807448       -79.199056   
1              -79.304302         KFC       43.780400       -79.300700   
2              -79.304302  A&W Canada       43.780327       -79.299060   
3              -79.318389         KFC       43.798938       -79.318854   
4              -79.318389  McDonald's       43.798249       -79.318167   

         Venue Category  
0  Fast Food Restaurant  
1  Fast Food Restaurant  
2  Fast Food Restaur

In [164]:
for x in range(len(fastFoodRestaurants['Venue'])):
  lat = fastFoodRestaurants['Venue Latitude'][x]
  lon = fastFoodRestaurants['Venue Longitude'][x]
  folium.Marker(location=[lat, lon],icon=folium.Icon(color='orange')).add_to(mapanalysis01)
  


mapanalysis01

#Results and Discussion

Our analysis show that over the map of Toronto we have a few oportunities to put on restaurant near paks, having more oportunities on te board of the city because the center of Toronto is a busy of restaurants

#Conclusion

Purpose of this project was to identify Toronto areas close to parks with low number of restaurants (particularly regular restaurants and fast food restaurants) in order to aid stakeholders in narrowing down the search for optimal location for a new  restaurant.

Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, but the analysis of map and locations showed in this report allow the processing of decision givin more information in in smart way.